In [ ]:
%aws configure

In [ ]:
%ssh -i "ColabOmkar3KeyPair.pem" ec2-user@ec2-98-92-23-236.compute-1.amazonaws.com

In [ ]:
%sudo dnf update -y

In [ ]:
%sudo dnf install -y docker

In [ ]:
%sudo systemctl start docker
%sudo systemctl enable docker
%sudo usermod -aG docker ec2-user

Exit & SSH again

In [ ]:
%docker --version
%docker ps

In [ ]:
%docker login nvcr.io
%Username: $oauthtoken
%Password: <your_token_here>

In [ ]:
%aws ecr get-login-password --region us-east-1 | docker login --username AWS --password-stdin 648758970526.dkr.ecr.us-east-1.amazonaws.com

In [ ]:
%docker pull nvcr.io/nvidia/tensorrt:24.02-py3
%docker tag nvcr.io/nvidia/tensorrt:24.02-py3 648758970526.dkr.ecr.us-east-1.amazonaws.com/tensorrt:24.02-py3
%docker images | grep tensorrt
%docker push 648758970526.dkr.ecr.us-east-1.amazonaws.com/tensorrt:24.02-py3

In [ ]:
%pip install torch torchvision nvidia-ml-py

In [ ]:
%conda install -c conda-forge onnx -y

Retrieving notices: done
Channels:
 - conda-forge
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.9.1
    latest version: 25.11.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



## Package Plan ##

  environment location: /home/ec2-user/anaconda3/envs/pytorch_p310

  added / updated specs:
    - onnx


The following packages will be UPDATED:

  ca-certificates                      2025.10.5-hbd8a1cb_0 --> 2025.11.12-hbd8a1cb_0 
  certifi                            2025.10.5-pyhd8ed1ab_0 --> 2025.11.12-pyhd8ed1ab_0 
  openssl                                  3.5.4-h26f9b46_0 --> 3.6.0-h26f9b46_0 




Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [5]:
import torch
import torchvision.models as models

model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
model.eval()

dummy_input = torch.randn(1, 3, 224, 224)

torch.onnx.export(
    model,
    dummy_input,
    "resnet50.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=17
)

print("Resnet ONNX Exported Successfully")

Resnet ONNX Exported Successfully


In [ ]:
%%writefile build_resnet50_trt.py
import tensorrt as trt
import os

TRT_LOGGER = trt.Logger(trt.Logger.INFO)

onnx_path = "/opt/ml/processing/input/resnet50.onnx"
engine_path = "/opt/ml/processing/output/resnet50.trt"

print("Loading ONNX model:", onnx_path)

trt.init_libnvinfer_plugins(TRT_LOGGER, "")

with trt.Builder(TRT_LOGGER) as builder, \
     builder.create_network(1 << int(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)) as network, \
     trt.OnnxParser(network, TRT_LOGGER) as parser:

    config = builder.create_builder_config()

    config.set_memory_pool_limit(trt.MemoryPoolType.WORKSPACE, 2 << 30)

    if builder.platform_has_fast_fp16:
        config.set_flag(trt.BuilderFlag.FP16)
        print("FP16 enabled")

    with open(onnx_path, "rb") as f:
        if not parser.parse(f.read()):
            for i in range(parser.num_errors):
                print(parser.get_error(i))
            raise RuntimeError("ONNX parsing failed")

    print("Building TensorRT engine...")
    engine_bytes = builder.build_serialized_network(network, config)

    if engine_bytes is None:
        raise RuntimeError("Failed To Build TensorRT Engine")

    with open(engine_path, "wb") as f:
        f.write(engine_bytes)

print("ResNet50 TensorRT Engine Built Successfully")
print("Saved to:", engine_path)

Writing build_resnet50_trt.py


In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput
import sagemaker

role = sagemaker.get_execution_role()

processor = ScriptProcessor(
    role=role,
    image_uri="648758970526.dkr.ecr.us-east-1.amazonaws.com/tensorrt:24.02-py3",
    command=["python3"],
    instance_count=1,
    instance_type="ml.g5.2xlarge",
    volume_size_in_gb=30,
    max_runtime_in_seconds=600,
)

processor.run(
    code="build_resnet50_trt.py",
    inputs=[
        ProcessingInput(
            source="resnet50.onnx",
            destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            source="/opt/ml/processing/output"
        )
    ]
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


INFO:sagemaker:Creating processing-job with name tensorrt-2025-12-14-01-12-29-276


...............[12/14/2025-01:14:57] [TRT] [I] [MemUsageChange] Init CUDA: CPU +18, GPU +0, now: CPU 25, GPU 253 (MiB)
[12/14/2025-01:15:03] [TRT] [I] [MemUsageChange] Init builder kernel library: CPU +1453, GPU +268, now: CPU 1555, GPU 521 (MiB)
[12/14/2025-01:15:03] [TRT] [I] Graph optimization time: 0.22248 seconds.
[12/14/2025-01:15:03] [TRT] [I] Local timing cache in use. Profiling results in this builder pass will not be stored.
[12/14/2025-01:15:35] [TRT] [I] Detected 1 inputs and 1 output network tensors.
[12/14/2025-01:15:36] [TRT] [I] Total Host Persistent Memory: 330096
[12/14/2025-01:15:36] [TRT] [I] Total Device Persistent Memory: 5120
[12/14/2025-01:15:36] [TRT] [I] Total Scratch Memory: 0
[12/14/2025-01:15:36] [TRT] [I] [MemUsageStats] Peak memory usage of TRT CPU/GPU memory allocators: CPU 48 MiB, GPU 49 MiB
[12/14/2025-01:15:36] [TRT] [I] [BlockAssignment] Started assigning block shifts. This will take 58 steps to complete.
[12/14/2025-01:15:36] [TRT] [I] [BlockAssignm

In [10]:
!aws s3 cp \
s3://sagemaker-us-east-1-648758970526/tensorrt-2025-12-14-01-12-29-276/output/output-1/resnet50.trt \
./resnet50.trt

download: s3://sagemaker-us-east-1-648758970526/tensorrt-2025-12-14-01-12-29-276/output/output-1/resnet50.trt to ./resnet50.trt


In [19]:
!wget -O dog.jpg https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg

--2025-12-14 01:38:54--  https://raw.githubusercontent.com/pytorch/hub/master/images/dog.jpg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 661378 (646K) [image/jpeg]
Saving to: ‘dog.jpg’

100%[======================================>] 661,378     --.-K/s   in 0.007s  

2025-12-14 01:38:54 (96.6 MB/s) - ‘dog.jpg’ saved [661378/661378]



In [20]:
%%writefile run_resnet50_trt_infer.py
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from PIL import Image

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

engine_path = "/opt/ml/processing/input/model/resnet50.trt"
image_path = "/opt/ml/processing/input/image/dog.jpg"

# Load engine
with open(engine_path, "rb") as f:
    engine = trt.Runtime(TRT_LOGGER).deserialize_cuda_engine(f.read())

context = engine.create_execution_context()

# Load & preprocess image
img = Image.open(image_path).convert("RGB")
img = img.resize((224, 224))

img = np.array(img).astype(np.float32) / 255.0
img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]

img = img.transpose(2, 0, 1)
img = np.expand_dims(img, axis=0)
img = np.ascontiguousarray(img)

# Buffers
h_input = img.astype(np.float32)
h_output = np.empty((1, 1000), dtype=np.float32)

d_input = cuda.mem_alloc(h_input.nbytes)
d_output = cuda.mem_alloc(h_output.nbytes)

cuda.memcpy_htod(d_input, h_input)
context.execute_v2([int(d_input), int(d_output)])
cuda.memcpy_dtoh(h_output, d_output)

pred = np.argmax(h_output, axis=1)[0]
print("Predicted ImageNet Class ID:", pred)

Writing run_resnet50_trt_infer.py


In [22]:
from sagemaker.processing import ScriptProcessor, ProcessingInput
import sagemaker

role = sagemaker.get_execution_role()

processor = ScriptProcessor(
    role=role,
    image_uri="648758970526.dkr.ecr.us-east-1.amazonaws.com/tensorrt:24.02-py3",
    command=["python3"],
    instance_count=1,
    instance_type="ml.g5.2xlarge",
    volume_size_in_gb=30,
    max_runtime_in_seconds=600,
)

processor.run(
    code="run_resnet50_trt_infer.py",
    inputs=[
        ProcessingInput(
            source="resnet50.trt",
            destination="/opt/ml/processing/input/model"
        ),
        ProcessingInput(
            source="dog.jpg",
            destination="/opt/ml/processing/input/image"
        ),
    ]
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating processing-job with name tensorrt-2025-12-14-01-40-32-769


...............Predicted ImageNet Class ID: 258



In [23]:
%%writefile trt_timing_resnet50.py
import time
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from PIL import Image

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

ENGINE_PATH = "/opt/ml/processing/input/model/resnet50.trt"
IMAGE_PATH = "/opt/ml/processing/input/image/dog.jpg"

# -------------------------
# Load TensorRT engine
# -------------------------
with open(ENGINE_PATH, "rb") as f:
    engine = trt.Runtime(TRT_LOGGER).deserialize_cuda_engine(f.read())

context = engine.create_execution_context()

# -------------------------
# Load & preprocess image
# -------------------------
img = Image.open(IMAGE_PATH).convert("RGB")
img = img.resize((224, 224))

img = np.array(img).astype(np.float32) / 255.0
img = (img - [0.485, 0.456, 0.406]) / [0.229, 0.224, 0.225]
img = img.transpose(2, 0, 1)
img = np.expand_dims(img, axis=0)
img = np.ascontiguousarray(img)

# -------------------------
# Allocate buffers
# -------------------------
h_input = img.astype(np.float32)
h_output = np.empty((1, 1000), dtype=np.float32)

d_input = cuda.mem_alloc(h_input.nbytes)
d_output = cuda.mem_alloc(h_output.nbytes)

# -------------------------
# Warm-up
# -------------------------
for _ in range(20):
    cuda.memcpy_htod(d_input, h_input)
    context.execute_v2([int(d_input), int(d_output)])
    cuda.memcpy_dtoh(h_output, d_output)

cuda.Context.synchronize()

# -------------------------
# Timing
# -------------------------
ITERATIONS = 200

start = time.time()
for _ in range(ITERATIONS):
    cuda.memcpy_htod(d_input, h_input)
    context.execute_v2([int(d_input), int(d_output)])
    cuda.memcpy_dtoh(h_output, d_output)

cuda.Context.synchronize()
elapsed = time.time() - start

avg_ms = (elapsed / ITERATIONS) * 1000

print(f"TensorRT Average Inference Time: {avg_ms:.3f} ms")

Writing trt_timing_resnet50.py


In [ ]:
from sagemaker.processing import ScriptProcessor, ProcessingInput
import sagemaker

role = sagemaker.get_execution_role()

processor = ScriptProcessor(
    role=role,
    image_uri="648758970526.dkr.ecr.us-east-1.amazonaws.com/tensorrt:24.02-py3",
    command=["python3"],
    instance_count=1,
    instance_type="ml.g5.2xlarge",
    volume_size_in_gb=30,
    max_runtime_in_seconds=600,
)

processor.run(
    code="trt_timing_resnet50.py",
    inputs=[
        ProcessingInput(source="resnet50.trt", destination="/opt/ml/processing/input/model"),
        ProcessingInput(source="dog.jpg", destination="/opt/ml/processing/input/image"),
    ]
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating processing-job with name tensorrt-2025-12-14-01-49-43-493


...........

...............
TensorRT Average Inference Time: 0.725 ms
Pytorch Inference Time: 5.4 ms